<a href="https://colab.research.google.com/github/meefs/RecursiveSummarizer/blob/main/RecursiveSummarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup environment

In [1]:
!pip install openai
!git clone https://github.com/meefs/RecursiveSummarizer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 KB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.4-py3-none-any.whl size=67744 sha256=f972786583717c638452492718e4ea6ab5df4900bab14a72078d7e3f55d84db7
  Stored in directory: /root/.cache/pip/wheels/2b/d8/4e/268f029bd3277c1dd9e8781a0e0296e0a63822665bfa2429fc
Successfully built openai
Cloning into 'RecursiveSummarizer'...
remote: Enumerating objects: 1050, done.
remote: Counting objects: 100% (1050/1050), done.
remote: Compressing objects: 100% (1014/1014), done.
remote: Total 1050 (delta 37), reused 1041 (delta 33), pack-reused 0
Receiving objects: 100% (1050/1050), 3.46 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/co

Run RecusiveSummarizer

In [14]:
import openai
import os
from time import time,sleep
import textwrap
import re


def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()


openai.api_key = open_file('openaiapikey.txt')


def save_file(content, filepath):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

engine = "text-davinci-002" #@param ["text-davinci-003", "text-davinci-002", "text-davinci-001", "text-curie-001", "text-babbage-001", "text-ada-001"]
temp = 0.75 #@param {type:"slider", min:0, max:1, step:0.01}
top_p = 1 #@param {type:"slider", min:0, max:1, step:0.1}
tokens = 3337 #@param {type:"slider", min:0, max:4000, step:1}
freq_pen = 0.25 #@param {type:"slider", min:-2, max:2, step:0.05}
pres_pen = 0.25 #@param {type:"slider", min:-2, max:2, step:0.05}
def gpt3_completion(prompt, engine=engine, temp=temp, top_p=top_p, tokens=tokens, freq_pen=freq_pen, pres_pen=pres_pen, stop=['<<END>>']):
    max_retry = 5
    retry = 0
    while True:
        try:
            response = openai.Completion.create(
                engine=engine,
                prompt=prompt,
                temperature=temp,
                max_tokens=tokens,
                top_p=top_p,
                frequency_penalty=freq_pen,
                presence_penalty=pres_pen,
                stop=stop)
            text = response['choices'][0]['text'].strip()
            text = re.sub('\s+', ' ', text)
            filename = '%s_gpt3.txt' % time()
            with open('gpt3_logs/%s' % filename, 'w') as outfile:
                outfile.write('PROMPT:\n\n' + prompt + '\n\n==========\n\nRESPONSE:\n\n' + text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3 error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)


if __name__ == '__main__':
    alltext = open_file('input.txt')
    chunks = textwrap.wrap(alltext, 2000)
    result = list()
    count = 0
    for chunk in chunks:
        count = count + 1
        prompt = open_file('prompt.txt').replace('<<SUMMARY>>', chunk)
        prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
        summary = gpt3_completion(prompt)
        print('\n\n\n', count, 'of', len(chunks), ' - ', summary)
        result.append(summary)
    save_file('\n\n'.join(result), 'output_%s.txt' % time())




 1 of 10  -  The essay is about a man reflecting on the life of Kidney Foot Cassidy, a man who was known for begging for money on the streets. The essayist reflects on how he used to see Cassidy around town, and how he won't be seeing him anymore now that he is dead.



 2 of 10  -  Kidney Foot Cassidy is a homeless man who befriends a fellow homeless man, Floyd, in Chicago. Floyd has a dime, which he offers to Kidney Foot in order to buy him a hot dog and a cup of coffee. However, Kidney Foot is only interested in drinking, not eating, and so he declines the offer.



 3 of 10  -  A man and his friend, "Kidney Foot", are walking down an alley when they are accosted by a man in a big coat. Kidney Foot takes the coat and two bottles of liquor from the man, and the man falls to the ground. The narrator, "Floyd", is surprised when Kidney Foot tells him that he killed the man. They take the coat and bottles back to Floyd's place, where Kidney Foot proceeds to drink most of the liquor.

